Project rundown



Things to complete in our FINAL PROJECT

library
Read and load the files
regression analysis (cross-validation, standard error, prediction, visualization)
written report on what the processes are
results
implications


plot for each variable separately plot_beds <- sacramento |> ggplot(aes(x = beds, y = price)) + geom_point(alpha = 0.4) + labs(x = 'Number of Bedrooms', y = 'Price (USD)') + theme(text = element_text(size = 12)) plot_beds

split data sacramento_split <- initial_split(sacramento, prop = 0.75, strata = price) sacramento_train <- training(sacramento_split) sacramento_test <- testing(sacramento_split)

create recipe, a specification for k sacr_recipe <- recipe(price ~ sqft + beds, data = sacramento_train) |> step_scale(all_predictors()) |> step_center(all_predictors())

sacr_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> set_engine("kknn") |> set_mode("regression")

5-fold cross-validation, choose the number of neighbors via the minimum RMSPE: sacr_vfold <- vfold_cv(sacramento_train, v = 5, strata = price) gridvals <- tibble(neighbors = seq(1, 200))
sacr_multi <- workflow() |> add_recipe(sacr_recipe) |> add_model(sacr_spec) |> tune_grid(sacr_vfold, grid = gridvals) |> collect_metrics() |> filter(.metric == "rmse") |> filter(mean == min(mean))

sacr_k <- sacr_multi |> pull(neighbors) sacr_multi

Choose the optimal K to run the full analyis on test set sacr_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = sacr_k) |> set_engine("kknn") |> set_mode("regression")
knn_mult_fit <- workflow() |> add_recipe(sacr_recipe) |> add_model(sacr_spec) |> fit(data = sacramento_train)

knn_mult_preds <- knn_mult_fit |> predict(sacramento_test) |> bind_cols(sacramento_test)

knn_mult_mets <- metrics(knn_mult_preds, truth = price, estimate = .pred) |> filter(.metric == 'rmse')

knn_mult_mets